In [ ]:
import tensorflow as tf
import numpy as np
from mlp.data_providers import CIFAR10DataProvider, CIFAR100DataProvider
import matplotlib.pyplot as plt
%matplotlib inline

## CIFAR-10 and CIFAR-100 datasets

[CIFAR-10 and CIFAR-100](https://www.cs.toronto.edu/~kriz/cifar.html) are a pair of image classification datasets collected by collected by Alex Krizhevsky, Vinod Nair, and Geoffrey Hinton. They are labelled subsets of the much larger [80 million tiny images](dataset). They are a common benchmark task for image classification - a list of current accuracy benchmarks for both data sets are maintained by Rodrigo Benenson [here](http://rodrigob.github.io/are_we_there_yet/build/).

As the name suggests, CIFAR-10 has images in 10 classes:

> airplane, automobile, bird, cat, deer, dog, frog, horse, ship, and truck

with 6000 images per class for an overall dataset size of 60000. Each image has three (RGB) colour channels and pixel dimension 32×32, corresponding to a total dimension per input image of 3×32×32=3072. For each colour channel the input values have been normalised to the range [0, 1].

CIFAR-100 has images of identical dimensions to CIFAR-10 but rather than 10 classes they are instead split across 100 fine-grained classes (and 20 coarser 'superclasses' comprising multiple finer classes):

<table style='border: none;'>
    <tbody><tr style='font-weight: bold;'>
        <td>Superclass</td>
        <td>Classes</td>
    </tr>
    <tr>
        <td>aquatic mammals</td>
        <td>beaver, dolphin, otter, seal, whale</td>
    </tr>
    <tr>
        <td>fish</td>
        <td>aquarium fish, flatfish, ray, shark, trout</td>
    </tr>
    <tr>
        <td>flowers</td>
        <td>orchids, poppies, roses, sunflowers, tulips</td>
    </tr>
    <tr>
        <td>food containers</td>
        <td>bottles, bowls, cans, cups, plates</td>
    </tr>
    <tr>
        <td>fruit and vegetables</td>
        <td>apples, mushrooms, oranges, pears, sweet peppers</td>
    </tr>
    <tr>
        <td>household electrical devices</td>
        <td>clock, computer keyboard, lamp, telephone, television</td>
    </tr>
    <tr>
        <td>household furniture</td>
        <td>bed, chair, couch, table, wardrobe</td>
    </tr>
    <tr>
        <td>insects</td>
        <td>bee, beetle, butterfly, caterpillar, cockroach</td>
    </tr>
    <tr>
        <td>large carnivores</td>
        <td>bear, leopard, lion, tiger, wolf</td>
    </tr>
    <tr>
        <td>large man-made outdoor things</td>
        <td>bridge, castle, house, road, skyscraper</td>
    </tr>
    <tr>
        <td>large natural outdoor scenes</td>
        <td>cloud, forest, mountain, plain, sea</td>
    </tr>
    <tr>
        <td>large omnivores and herbivores</td>
        <td>camel, cattle, chimpanzee, elephant, kangaroo</td>
    </tr>
    <tr>
        <td>medium-sized mammals</td>
        <td>fox, porcupine, possum, raccoon, skunk</td>
    </tr>
    <tr>
        <td>non-insect invertebrates</td>
        <td>crab, lobster, snail, spider, worm</td>
    </tr>
    <tr>
        <td>people</td>
        <td>baby, boy, girl, man, woman</td>
    </tr>
    <tr>
        <td>reptiles</td>
        <td>crocodile, dinosaur, lizard, snake, turtle</td>
    </tr>
    <tr>
        <td>small mammals</td>
        <td>hamster, mouse, rabbit, shrew, squirrel</td>
    </tr>
    <tr>
        <td>trees</td>
        <td>maple, oak, palm, pine, willow</td>
    </tr>
    <tr>
        <td>vehicles 1</td>
        <td>bicycle, bus, motorcycle, pickup truck, train</td>
    </tr>
    <tr>
        <td>vehicles 2</td>
        <td>lawn-mower, rocket, streetcar, tank, tractor</td>
    </tr>
</tbody></table>

Each class has 600 examples in it, giving an overall dataset size of 60000 i.e. the same as CIFAR-10.

Both CIFAR-10 and CIFAR-100 have standard splits into 50000 training examples and 10000 test examples. To avoid accidental (or purposeful...) fitting to the test set, we have used a different assignation of examples to test and training sets and only provided the inputs (and not target labels) for the 10000 examples chosen for the test set. The remaining 50000 examples have been split in to a 40000 example training dataset and a 10000 example validation dataset, each with target labels provided. If you wish to use a more complex cross-fold validation scheme you may want to combine these two portions of the dataset and define your own functions for separating out a validation set.

Data provider classes for both CIFAR-10 and CIFAR-100 are available in the `mlp.data_providers` module. Both have similar behaviour to the `MNISTDataProvider` used extensively last semester. A `which_set` argument can be used to specify whether to return a data provided for the training dataset (`which_set='train'`) or validation dataset (`which_set='valid'`). 

The CIFAR-100 data provider also takes an optional `use_coarse_targets` argument in its constructor. By default this is set to `False` and the targets returned by the data provider correspond to 1-of-K encoded binary vectors for the 100 fine-grained object classes. If `use_coarse_targets=True` then instead the data provider will return 1-of-K encoded binary vector targets for the 20 coarse-grained superclasses associated with each input instead.

Both data provider classes provide a `label_map` attribute which is a list of strings which are the class labels corresponding to the integer targets (i.e. prior to conversion to a 1-of-K encoded binary vector).

Below example code is given for creating instances of the CIFAR-10 and CIFAR-100 data provider objects and using them to train simple two-layer feedforward network models with rectified linear activations in TensorFlow. You may wish to use this code as a starting point for your own experiments.


### Accessing the CIFAR-10 and CIFAR-100 data

Before using the data provider objects you will need to copy the associated data files in to your local `mlp/data` directory (or wherever your `MLP_DATA_DIR` environment variable points to if different). The data is available as six compressed NumPy `.npz` files, (`cifar-10-train.npz, cifar-10-valid.npz, cifar-10-test-inputs.npz` and `cifar-100-train.npz, cifar-100-valid.npz, cifar-100-test.npz`) in the AFS directory `/afs/inf.ed.ac.uk/group/teaching/mlp/data`. Assuming your local `mlpractical` repository is in your home directory you should be able to copy the required files by running

```
cp /afs/inf.ed.ac.uk/group/teaching/mlp/data/cifar*.npz ~/mlpractical/data
```

## Example two-layer classifier model on CIFAR-10

In [ ]:
train_data = CIFAR10DataProvider('train', batch_size=50)
valid_data = CIFAR10DataProvider('valid', batch_size=50)

In [ ]:
def fully_connected_layer(inputs, input_dim, output_dim, nonlinearity=tf.nn.relu):
    weights = tf.Variable(
        tf.truncated_normal(
            [input_dim, output_dim], stddev=2. / (input_dim + output_dim)**0.5), 
        'weights')
    biases = tf.Variable(tf.zeros([output_dim]), 'biases')
    outputs = tf.matmul(inputs, weights) + biases
    return outputs

In [ ]:
inputs = tf.placeholder(tf.float32, [None, train_data.inputs.shape[1]], 'inputs')
targets = tf.placeholder(tf.float32, [None, train_data.num_classes], 'targets')
num_hidden = 200

with tf.name_scope('fc-layer-1'):
    hidden_1 = fully_connected_layer(inputs, train_data.inputs.shape[1], num_hidden)
with tf.name_scope('output-layer'):
    outputs = fully_connected_layer(hidden_1, num_hidden, train_data.num_classes, tf.identity)

with tf.name_scope('error'):
    error = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(outputs, targets))
with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(
            tf.equal(tf.argmax(outputs, 1), tf.argmax(targets, 1)), 
            tf.float32))

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer().minimize(error)
    
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for e in range(25):
        running_error = 0.
        running_accuracy = 0.
        for input_batch, target_batch in train_data:
            _, batch_error, batch_acc = sess.run(
                [train_step, error, accuracy], 
                feed_dict={inputs: input_batch, targets: target_batch})
            running_error += batch_error
            running_accuracy += batch_acc
        running_error /= train_data.num_batches
        running_accuracy /= train_data.num_batches
        print('End of epoch {0:02d}: err(train)={1:.2f} acc(train)={2:.2f}'
              .format(e + 1, running_error, running_accuracy))
        if (e + 1) % 5 == 0:
            valid_error = 0.
            valid_accuracy = 0.
            for input_batch, target_batch in valid_data:
                batch_error, batch_acc = sess.run(
                    [error, accuracy], 
                    feed_dict={inputs: input_batch, targets: target_batch})
                valid_error += batch_error
                valid_accuracy += batch_acc
            valid_error /= valid_data.num_batches
            valid_accuracy /= valid_data.num_batches
            print('                 err(valid)={0:.2f} acc(valid)={1:.2f}'
                   .format(valid_error, valid_accuracy))

## Example two-layer classifier model on CIFAR-100

In [ ]:
train_data = CIFAR100DataProvider('train', batch_size=50)
valid_data = CIFAR100DataProvider('valid', batch_size=50)

In [ ]:
tf.reset_default_graph()

inputs = tf.placeholder(tf.float32, [None, train_data.inputs.shape[1]], 'inputs')
targets = tf.placeholder(tf.float32, [None, train_data.num_classes], 'targets')
num_hidden = 200

with tf.name_scope('fc-layer-1'):
    hidden_1 = fully_connected_layer(inputs, train_data.inputs.shape[1], num_hidden)
with tf.name_scope('output-layer'):
    outputs = fully_connected_layer(hidden_1, num_hidden, train_data.num_classes, tf.identity)

with tf.name_scope('error'):
    error = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(outputs, targets))
with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(
            tf.equal(tf.argmax(outputs, 1), tf.argmax(targets, 1)), 
            tf.float32))

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer().minimize(error)
    
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for e in range(25):
        running_error = 0.
        running_accuracy = 0.
        for input_batch, target_batch in train_data:
            _, batch_error, batch_acc = sess.run(
                [train_step, error, accuracy], 
                feed_dict={inputs: input_batch, targets: target_batch})
            running_error += batch_error
            running_accuracy += batch_acc
        running_error /= train_data.num_batches
        running_accuracy /= train_data.num_batches
        print('End of epoch {0:02d}: err(train)={1:.2f} acc(train)={2:.2f}'
              .format(e + 1, running_error, running_accuracy))
        if (e + 1) % 5 == 0:
            valid_error = 0.
            valid_accuracy = 0.
            for input_batch, target_batch in valid_data:
                batch_error, batch_acc = sess.run(
                    [error, accuracy], 
                    feed_dict={inputs: input_batch, targets: target_batch})
                valid_error += batch_error
                valid_accuracy += batch_acc
            valid_error /= valid_data.num_batches
            valid_accuracy /= valid_data.num_batches
            print('                 err(valid)={0:.2f} acc(valid)={1:.2f}'
                   .format(valid_error, valid_accuracy))